In [6]:
import os
import random
from openai import OpenAI

client = OpenAI(
    api_key="sk-aeaf628804b14d708945643370bcf5c4",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 读取违规方向
with open("reasons.txt", "r", encoding="utf-8") as file:
    directions = [line.strip() for line in file if line.strip()]

# 隐私政策文件列表（假设路径为policies/下）
policy_files = [f"{i}.txt" for i in range(1, 31)]
output_dir = "generated"
os.makedirs(output_dir, exist_ok=True)

# 每轮请求的修改方向数量
batch_sizes = [1, 1, 1, 2, 5]

for policy_path in policy_files:
    # 读取隐私政策内容
    with open(policy_path, "r", encoding="utf-8") as f:
        original_lines = f.readlines()

    # 打乱方向并分配批次
    shuffled_directions = directions.copy()
    random.shuffle(shuffled_directions)
    pointer = 0
    selected_batches = []
    for size in batch_sizes:
        batch = shuffled_directions[pointer:pointer+size]
        selected_batches.append(batch)
        pointer += size
    
    # 初始化位置记录
    position_records = []
    total_new_clauses = []
    user_query = f"<policy>{original_lines}</policy>"
    # 为每个批次生成文件
    for idx, (size, batch) in enumerate(zip(batch_sizes, selected_batches), 1):
        # 构造用户提示
        new_clauses = []
        print(size)
        for i in range(size):
        # 调用API生成内容
            response = client.chat.completions.create(
                model="qwen-turbo",
                messages=[
                    {"role": "system", "content": f"Here is a user privacy policy of an app. You are expected to generate one more improper article which violates the GDPR for the given reason: \[ \"{batch[i]}\" \]. (You should not directly adapt the given example but learn from it.)  Your generated article should have related content with the given policy below and your article should be concise.  You are expected to try to hide the flaw of your generated article. Just reply the generated article sentence with no title or other things. Do not say any extra thing."},
                    {"role": "user", "content": user_query}
                ]
            )
            generated = response.choices[0].message.content
            print(generated)
            new_clauses.append(generated)
            total_new_clauses.append(generated)
        # 为每个新条款选择插入位置
        modified_lines = original_lines.copy()
        insertion_positions = []
        
        for clause in new_clauses:
            # 随机选择插入位置（0到行数之间）
            insert_pos = random.randint(0, len(modified_lines))
            modified_lines.insert(insert_pos, clause + "\n")
            insertion_positions.append(insert_pos)
            for i in range(len(insertion_positions)):
                if insert_pos <= insertion_positions[i]:
                    insertion_positions[i] = insertion_positions[i] + 1
        
        # 记录本批次插入位置
        position_records.append(",".join(map(str, insertion_positions)))
        
        # 生成文件名
        base_name = os.path.splitext(os.path.basename(policy_path))[0]
        output_path = os.path.join(
            output_dir,
            f"{base_name}_gene{idx}_{len(batch)}.txt"
        )
        
        # 保存修改后的文件
        with open(output_path, "w", encoding="utf-8") as f:
            f.writelines(modified_lines)

    # 生成位置记录文件（第6个文件）
    print("pos")
    pos_output_path = os.path.join(
        output_dir,
        f"{base_name}_genex_pos.txt"
    )
    with open(pos_output_path, "w", encoding="utf-8") as f:
        f.write("\n".join(position_records))
    # 生成total文件（第7个文件）
    total_pos = []
    total_position_records = []
    print("total")
    modified_lines = original_lines.copy()
    for clause in total_new_clauses:
        insert_pos = random.randint(0, len(modified_lines))
        modified_lines.insert(insert_pos, clause + "\n")
        for i in range(len(total_pos)):
            if insert_pos <= total_pos[i]:
                total_pos[i] = total_pos[i] + 1
        total_pos.append(insert_pos)
    total_position_records.append(",".join(map(str, total_pos)))
    base_name = os.path.splitext(os.path.basename(policy_path))[0]
    output_path = os.path.join(
        output_dir,
        f"{base_name}_genex_total.txt"
    )
    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(modified_lines)
    # 生成total_pos文件（第8个文件）
    print("total_pos")
    pos_output_path = os.path.join(
        output_dir,
        f"{base_name}_genex_total_pos.txt"
    )
    with open(pos_output_path, "w", encoding="utf-8") as f:
        f.write("\n".join(total_position_records))

1
"Our application utilizes advanced analytical techniques and complex system frameworks to meticulously gather user interaction data, ensuring the comprehensive integration of pertinent behavioral insights across various digital platforms associated with your device."
1
The app collects the user's daily step count to suggest personalized fitness goals, assuming higher steps correlate with improved health outcomes.
1
If you request to delete your data, we will honor your request but due to the extensive manual process involved in identifying and removing all instances of your data across our systems, it may take up to 180 days to complete the deletion.
2
User data may be transferred to third-party data centers in regions with lower data protection standards when local storage capacity is insufficient.
We will collect the browsing history of the user to optimize the color scheme of the app interface.
5
BlackBerry Limited processes your personal information for its internal business purp